# Example: Three spings in series

In [1]:
using JuAFEM

## Input

In [2]:
k = 1.0;   # Spring stiffness [N/m]
F = 1.0;   # Load magnitude [N]

1.0

### Element properties

In [3]:
ep1 = k;    # Stiffness for spring 1 etc.
ep2 = 2*k;   
ep3 = k;

## Geometry

### Element definitions - degrees of freedom (dofs)

In [4]:
# Note the transpose in the end
Edof = [1  1 2                 # Element 1: node 1->2
        2  2 3                 # Element 2: node 2->3
        3  3 4]';              # Element 3: node 3->4

3x3 Array{Int64,2}:
 1  2  3
 1  2  3
 2  3  4

### Boundary conditions - bc

In [5]:
bc = [1  0                      # Degrees of freedom 1 and 3 are prescribed to zero
      4  0];

## Stiffness matrix and load vector

In [6]:
K = zeros(4,4);      # Allocate space 
f = zeros(4,1);    

f[3] = F;             # Apply load in dof 2
 
Ke1 = spring1e(ep1);  # Element stiffness matrices
Ke2 = spring1e(ep2);
Ke3 = spring1e(ep3);

### Assemble element stiffnesses into global stiffness matrix

In [8]:
K = assemble(Edof[:,1], K, Ke1)
K = assemble(Edof[:,2], K, Ke2) 
K = assemble(Edof[:,3], K, Ke3) 
print(K)

[1.0 -1.0 0.0 0.0
 -1.0 3.0 -2.0 0.0
 0.0 -2.0 3.0 -1.0
 0.0 0.0 -1.0 1.0]

## Solve the equation system Ka = f 

In [10]:
(a, fb) = solveq(K, f, bc)      # a - vector of computed displacements
                                # fb - vector of computed boundary forces (zero for unconstrained dofs)
println("Displacement vector: a = $a")
println("Boundary forces:  = $fb")

Displacement vector: a = [0.0,0.39999999999999997,0.6,0.0]
Boundary forces:  = [-0.39999999999999997
 0.0
 -1.1102230246251565e-16
 -0.6]


## Compute sectional forces

In [13]:
el_disp = extract(Edof, a);     # Matrix contining the displacement vectors for each element 


es1 = spring1s(ep1, el_disp[:,1])        # Compute spring force in each element
es2 = spring1s(ep2, el_disp[:,2])
es3 = spring1s(ep3, el_disp[:,3])

print("Sectional forces are: N1 = $es1, N2 = $es2, N3 = $es3")

Sectional forces are: N1 = 0.39999999999999997, N2 = 0.4, N3 = -0.6